In [35]:
%run params.py

import pandas as pd
from plotting_functions import *
import numpy as np


lobref = pd.read_csv(os.path.join(DATAPATH, "ref", "lobstr_v3.0.2_hg19_ref_nochr.bed.gz"), usecols=range(4),
                     names=["chrom","start","end","period"], sep="\t")

data = LoadMLData(os.path.join(DATAPATH, "autosomal_estimates/perlocus", "autosomal_estimates_ml_filtered_v2.bed.gz"))
sdata = pd.read_csv(os.path.join(DATAPATH, "autosomal_estimates/perlocus", "autosomal_stutter_ml_v2.bed.gz"), names=["chrom","start","end","up","down","p"], sep="\t")
zdata = pd.read_csv(os.path.join(DATAPATH, "constraint", "autosomal_perlocus_estimates.bed"))

In [48]:
data["chrom"] = data["chrom"].apply(str)
data["start"] = data["start"].apply(int)
data["end"] = data["end"].apply(int)
sdata["chrom"] = sdata["chrom"].apply(str)
sdata["start"] = sdata["start"].apply(int)
sdata["end"] = sdata["end"].apply(int)
zdata["chrom"] = zdata["chrom"].apply(str)
zdata["start"] = zdata["start"].apply(int)
zdata["end"] = zdata["end"].apply(int)
lobref["chrom"] = lobref["chrom"].apply(str)
lobref["start"] = lobref["start"].apply(int)
lobref["end"] = lobref["end"].apply(int)

In [50]:
suppdata = pd.merge(data, sdata, on=["chrom","start","end"])
suppdata = pd.merge(suppdata, lobref, on=["chrom","start","end"])
suppdata = pd.merge(suppdata, zdata, on=["chrom","start","end"], how="outer")

In [51]:
# Small changes
suppdata["#chrom"] = suppdata["chrom"]
suppdata["numsamples_ml"] = suppdata["numsamples_ml"].apply(int)

In [53]:
columns = ["#chrom","start","end","est_logmu_ml","est_beta_eff_ml","est_beta_ml", \
          "est_pgeom_ml", "stderr_ml", "numsamples_ml", \
          "up", "down", "p", \
          "period", "motif","uninterrupted_length", \
          "pred_mu_1", "pred_mu_se_1", "zscore_1", \
          "pred_mu_2", "pred_mu_se_2", "zscore_2", "filter1"]
suppdata[columns].to_csv(os.path.join(DATAPATH, "suppdata", "Gymrek_etal_SupplementalData1.bed"), sep="\t", index=False)

In [54]:
suppdata.shape

(1251510, 30)